In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/test.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/quora-question-pairs/train.csv.zip")
print(train.shape)
test=pd.read_csv("/kaggle/input/quora-question-pairs/test.csv")
print(test.shape)

(404290, 6)
(2345796, 3)


In [3]:

test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [4]:
# since there is 1 null value in question 1 and two null values in question 2 we change it to empty string 
train['question1']=train['question1'].fillna('')
train['question2']=train['question2'].fillna('')
y_train=train['is_duplicate']
train=train.drop(['is_duplicate'],axis=1)

 

In [5]:
#for test 1
test['question1']=test['question1'].fillna('')
test['question2']=test['question2'].fillna('')


 

In [6]:
# adding related frequencies of the questions
 
train['freq_qid1'] =train.groupby('qid1')['qid1'].transform('count') 
train['freq_qid2']=train.groupby('qid2')['qid2'].transform('count')

# adding length of question 1 and question 2 
train['len_q1']=train['question1'].str.len()
train['len_q2']=train['question2'].str.len()



 

In [7]:
# adding number of words in each questions
train['n_words_q1']=train['question1'].apply(lambda x: len(x.split(" ")))
train['n_words_q2']=train['question2'].apply(lambda x: len(x.split(" ")))

In [8]:
# adding number of words in each questions of test
test['n_words_q1']=test['question1'].apply(lambda x: len(x.split(" ")))
test['n_words_q2']=test['question2'].apply(lambda x: len(x.split(" ")))

In [9]:
# length of common words in two questions
def len_common_words(data):
    w1=set(map(lambda x: x.lower().strip(),data['question1'].split(" ")))
    w2=set(map(lambda x: x.lower().strip(),data['question2'].split(" ")))
    return 1.0* len(w1 & w2) 
train['len_c-woords']=train.apply(len_common_words,axis=1) 
# length of unique common words only
def combined_words(data):
    w1=set(map(lambda x: x.lower().strip(),data['question1'].split(" ")))
    w2=set(map(lambda x: x.lower().strip(),data['question2'].split(" ")))
    return 1.0* (len(w1)+ len(w2) )
train['combined_words']=train.apply(combined_words,axis=1) 

# words share is length of uniquewords divided by total combined words 
def words_share(data):
    w1=set(map(lambda x: x.lower().strip(),data['question1'].split(" ")))
    w2=set(map(lambda x: x.lower().strip(),data['question2'].split(" ")))
    return 1.0* (len(w1 & w2)/(len(w1)+ len(w2) ))
train['words_share']=train.apply(words_share,axis=1) 

# frequency of question1 + frequency of question
train['freq_q1q2']=train['freq_qid1'] + train['freq_qid2'] 

# absolute value of (question 1' frequency - question 2's frequency)

train['abs_diff']=abs(train['freq_qid1'] -train['freq_qid2'] )

In [10]:
#for test
test['len_c-woords']=test.apply(len_common_words,axis=1) 
test['combined_words']=test.apply(combined_words,axis=1) 
test['words_share']=test.apply(words_share,axis=1) 


In [11]:
train.head()


,id,qid1,qid2,question1,question2,freq_qid1,freq_qid2,len_q1,len_q2,n_words_q1,n_words_q2,len_c-woords,combined_words,words_share,freq_q1q2,abs_diff
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [12]:
test.head(5)

,test_id,question1,question2,n_words_q1,n_words_q2,len_c-woords,combined_words,words_share
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,11,14,2.0,24.0,0.083333
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,14,7,4.0,21.0,0.190476
2,2,What but is the best way to send money from Ch...,What you send money to China?,14,6,4.0,18.0,0.222222
3,3,Which food not emulsifiers?,What foods fibre?,4,3,0.0,7.0,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,4,6,3.0,10.0,0.300000


In [13]:
import re
from nltk.stem.porter import *
from bs4 import BeautifulSoup
import nltk

from nltk.corpus import stopwords
stopwords=stopwords.words("english")

def text_preprocess(df):
    cleaned=df['question1'].apply(lambda x :[ i for i in x if i not in stopwords])
    return cleaned


In [14]:
import string
from bs4 import BeautifulSoup
train['question1'] = train['question1'].str.replace(r'[^\w\s]+', '')
 
train['question2']=train['question2'].str.replace(r'[^\w\s]+', '')

# question = BeautifulSoup(train['question1'])
# train['question1'] = question.get_text()

# question2 = BeautifulSoup(train['question2'])
# train['question2'] = question2.get_text()
train['question1'] = [BeautifulSoup(text).get_text() for text in train['question1'] ]
train['question2'] = [BeautifulSoup(text).get_text() for text in train['question2'] ]

In [15]:
#for test
test['question1'] = test['question1'].str.replace(r'[^\w\s]+', '')
 
test['question2']=test['question2'].str.replace(r'[^\w\s]+', '')

test['question1'] = [BeautifulSoup(text).get_text() for text in test['question1'] ]
test['question2'] = [BeautifulSoup(text).get_text() for text in test['question2'] ]


In [16]:
from nltk.stem import PorterStemmer
def stem_sentences(sentence):
    porter_stemmer = PorterStemmer()
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

train['question1'] = train['question1'].apply(stem_sentences)
train['question2'] = train['question2'].apply(stem_sentences)


In [17]:
#for test
test['question1'] = test['question1'].apply(stem_sentences)
test['question2'] = test['question2'].apply(stem_sentences)

In [18]:
#Now performing advanced featurizations 
def featurization(q1,q2):
    features=[0,0,0,0,0,0,0,0]
    q1=q1.split()
    q2=q2.split()
    
    if len(q1) == 0 or len(q2) == 0:
        return features
    
    #without stop words
    q1_nostop=set([each for each in q1 if each not in stopwords])
    q2_nostop=set([each for each in q2 if each not in stopwords])
    
    #with stop words
    q1_stop=set([each for each in q1 if each in stopwords])
    q2_stop=set([each for each in q2 if each in stopwords])
    #common non-stop words
    common_non_stop_c=len(q1_nostop.intersection(q2_nostop))
    
    #length of common stop words
    common_stop_c=len(q1_stop.intersection(q2_stop))
    
    # len of common unique words count from all together
    common_all_c=len(set(q1).intersection(set(q2)))
    
    # cmin,cmax for without stop words
    cmin_nostop=common_non_stop_c/(min(len(q1_nostop),len(q2_nostop))+0.0000001)
    cmax_nostop=common_non_stop_c/(max(len(q1_nostop),len(q2_nostop))+0.0000001)
    
    #cmin,cmax with stop words
    cmin_stop=common_stop_c/(min(len(q1_stop),len(q2_stop)) +0.0000001)
    cmax_stop=common_stop_c/(max(len(q1_stop),len(q2_stop))+0.0000001)
    
    #cminall,cmaxall
    cmin_all=common_all_c/(min(len(q1),len(q2))+0.0000001)
    cmax_all=common_all_c/(max(len(q1),len(q2))+0.0000001)
    
    first_same=int(q1[-1]==q2[-1])
    last_same=int(q1[0]==q2[0])
    
    
    features[0]=cmin_nostop
    features[1]=cmax_nostop
    features[2]=cmin_stop
    features[3]=cmax_stop
    features[4]=cmin_all
    features[5]=cmax_all
    features[6]=first_same
    features[7]=last_same
    return features
    
    
    

In [19]:
from fuzzywuzzy import fuzz

In [20]:

def get_features(dataframe):
    
    token_features = dataframe.apply(lambda x: featurization(x["question1"], x["question2"]), axis=1)
    
    dataframe["cmin_nostop"]    = list(map(lambda x: x[0], token_features))
    dataframe["cmax_nostop"]   = list(map(lambda x: x[1], token_features))
    dataframe["cmin_stop"]     = list(map(lambda x: x[2], token_features))
    dataframe["cmax_stop"]      = list(map(lambda x: x[3], token_features))
    dataframe["cmin_all"]      = list(map(lambda x: x[4], token_features))
    dataframe["cmax_all"] = list(map(lambda x: x[5], token_features))
    dataframe["first_same"]  = list(map(lambda x: x[6], token_features))
    dataframe["last_same"] = list(map(lambda x: x[7], token_features))
    
    dataframe["fuzz_ratio"]            = dataframe.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    dataframe["token_set_ratio"]       = dataframe.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    dataframe["token_sort_ratio"]      = dataframe.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    
    
    return dataframe


train=get_features(train)

 
    

In [21]:
train.head()


,id,qid1,qid2,question1,question2,freq_qid1,freq_qid2,len_q1,len_q2,n_words_q1,...,cmax_nostop,cmin_stop,cmax_stop,cmin_all,cmax_all,first_same,last_same,fuzz_ratio,token_set_ratio,token_sort_ratio
0,0,1,2,what is the step by step guid to invest in sha...,what is the step by step guid to invest in sha...,1,1,66,57,14,...,0.833333,1.000000,1.000000,0.916667,0.785714,0,1,92,100,92
1,1,3,4,what is the stori of kohinoor kohinoor diamond,what would happen if the indian govern stole t...,4,1,51,88,8,...,0.250000,0.666667,0.500000,0.500000,0.307692,0,1,66,81,64
2,2,5,6,how can I increas the speed of my internet con...,how can internet speed be increas by hack thro...,1,1,73,59,14,...,0.428571,0.400000,0.285714,0.500000,0.357143,0,1,54,73,66
3,3,7,8,whi am I mental veri lone how can I solv it,find the remaind when math2324math is divid by...,1,1,50,65,11,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,34,35,36
4,4,9,10,which one dissolv in water quikli sugar salt m...,which fish would surviv in salt water,3,1,76,39,13,...,0.200000,1.000000,0.666667,0.571429,0.307692,0,1,45,68,49


In [22]:
test=get_features(test)


In [23]:
test.head(5)

,test_id,question1,question2,n_words_q1,n_words_q2,len_c-woords,combined_words,words_share,cmin_nostop,cmax_nostop,cmin_stop,cmax_stop,cmin_all,cmax_all,first_same,last_same,fuzz_ratio,token_set_ratio,token_sort_ratio
0,0,how doe the surfac pro himself 4 compar with i...,whi did microsoft choos core m3 and not core i...,11,14,2.0,24.0,0.083333,0.500000,0.300000,0.000000,0.000000,0.272727,0.214286,0,0,42,57,54
1,1,should I have a hair transplant at age 24 how ...,how much cost doe hair transplant requir,14,7,4.0,21.0,0.190476,0.666667,0.500000,1.000000,0.166667,0.714286,0.357143,0,0,50,84,60
2,2,what but is the best way to send money from ch...,what you send money to china,14,6,4.0,18.0,0.222222,1.000000,0.500000,0.666667,0.333333,0.833333,0.357143,0,1,60,92,55
3,3,which food not emulsifi,what food fibr,4,3,0.0,7.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.333333,0.250000,0,0,54,54,54
4,4,how aberystwyth start read,how their can I start read,4,6,3.0,10.0,0.300000,0.666667,0.666667,1.000000,0.333333,0.750000,0.500000,1,1,65,70,62


In [24]:
total= list(train['question1']) + list(train['question2'])
#print(total)

In [25]:
#for test
total1=list(test['question1']) + list(test['question2'])

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
vectorizer.fit_transform(total)
print('-'*50)
print(vectorizer.idf_)
print('-'*50)


--------------------------------------------------
[10.69101313 11.96397881 12.81127667 ... 13.50442385 13.90988896
 13.50442385]
--------------------------------------------------


In [27]:
#for test
vectorizer1= TfidfVectorizer()
vectorizer1.fit_transform(test)
print('-'*50)
print(vectorizer1.idf_)
print('-'*50)

--------------------------------------------------
[3.30258509 3.30258509 3.30258509 3.30258509 3.30258509 3.30258509
 3.30258509 3.30258509 3.30258509 3.30258509 3.30258509 3.30258509
 3.30258509 3.30258509 3.30258509 3.30258509 3.30258509 3.30258509
 3.30258509 3.30258509]
--------------------------------------------------


In [28]:
q1vec=vectorizer.transform(train['question1'])
q2vec=vectorizer.transform(train['question2'])

In [29]:
#for test
q11vec=vectorizer1.transform(test['question1'])
q22vec=vectorizer1.transform(test['question2'])

In [30]:
train=train.drop(['question1','question2'],axis=1)

In [31]:
#for test
test=test.drop(['question1','question2'],axis=1)

In [32]:
from scipy.sparse import hstack 
final=hstack((train,q1vec,q2vec),format='csr', dtype='float64')

In [33]:
#for test
final1=hstack((test,q11vec,q22vec),format='csr', dtype='float64')

In [34]:
print(type(final))
print(final.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(404290, 171617)


In [35]:
#for test
print(type(final1))
print(final1.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(2345796, 57)


In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(random_state=0)
model.fit(train,y_train)
predicted= model.predict(train)
score= accuracy_score(predicted,y_train)
print(score)

0.999992579583962


In [38]:
from sklearn.metrics import log_loss
loss=log_loss(predicted,y_train)
print("The log loss is : ", loss)

The log loss is :  0.0002562980236400272
